<a href="https://colab.research.google.com/github/Semalign/Semalign/blob/main/MLusingPythonplusGPTMainly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ML model to predict car price

In [1]:
# now it is time to build my own ml model
import numpy as np
import pandas as pd

n_s=1000

# features include year,brand,mileage,cusomer_number
# targert is price

np.random.seed(3)

year=np.random.randint(2000,2024,n_s)
brand=np.random.choice(['A','B','C'],n_s)
mileage=np.random.randint(5000,200000,n_s)
c_n=np.random.randint(1,1000,n_s)

prices=((c_n*100)+(mileage*5)+(50000+(year%2000)*1000) + np.random.randint(-5000,5000,n_s))
# feature defintion finished here

car_data=pd.DataFrame({
    'year':year,
    'brand':brand,
    'mileage':mileage,
    'c_n':c_n,
    'price':prices,
})

car_data.to_csv('car_dat_synthetic.csv',index=False)



In [6]:
# next step is preparing the data for training
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

data=pd.read_csv('car_dat_synthetic.csv')

X=data.drop('price',axis=1)
y=data['price']

numerical_features=['year','mileage','c_n']
categorical_features=['brand']

numeric_transformer=Pipeline(steps=[('scaler',StandardScaler())])
categorical_transformer=Pipeline(steps=[('onehot',OneHotEncoder())])

preprocessor=ColumnTransformer(transformers=[
    ('num',numeric_transformer , numerical_features),
    ('cat',categorical_transformer,categorical_features)
])
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=10)

X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

# here we are done with preparing the data


In [8]:
# the next step is to chose a model and train the it
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

rf_model=RandomForestRegressor(n_estimators=100,random_state=10)
rf_model.fit(X_train,y_train)

y_pred=rf_model.predict(X_test)

mse=mean_squared_error(y_test,y_pred)

In [13]:
# now let's make the model more reliable
from sklearn.model_selection import GridSearchCV
param_grid={
    'n_estimators':[50,100,200],
    'max_depth':[None,10,20,30],
    'min_samples_split':[2,5,10],
    'min_samples_leaf':[1,2,4],
}

grid_search=GridSearchCV(estimator=RandomForestRegressor(random_state=10),param_grid=param_grid,cv=5,scoring='neg_mean_squared_error',n_jobs=-1)

grid_search.fit(X_train,y_train)

best_params=grid_search.best_params_
print(f'{best_params}')
best_rf_model = grid_search.best_estimator_
best_rf_y_pred = best_rf_model.predict(X_test)
best_rf_mse = mean_squared_error(y_test, best_rf_y_pred)
print(f'Best Random Forest Mean Squared Error: {best_rf_mse}')


{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
